# Music Listening Habits Data Mining

In [2]:
import pandas as pd
import os
import json
import requests
import ast


## Data needs to support vizualisation

- Top 15 artists (name & link) ✅
- Listening duration per artist ✅
- Number of different tracks listened per artist ✅

- Top 15 genres for each user ✅
- Listening time for each genre ✅
- Proportion of listening per genre ✅

- Average listening time per user ✅

- Ranking of genres per period (each week, each months)
- Ranking of tracks per period (each week, each months)✅
- Ranking of artist per period  (each week, each months)✅

- Exact listening time per user per date

## Import Data from Deezer

In [30]:
df_clement = pd.read_excel("./data/Archive-Clement-Deezer/clement-deezer-data.xlsx", "10_listeningHistory")

### Data Processing

In [40]:
# Drop useless columns
df_clement = df_clement[["Date", "Song Title", "Artist", "Album Title", "Listening Time"]]

# Drop negative listening time
df_clement = df_clement[df_clement["Listening Time"] > 0]

df_clement['Date'] = pd.to_datetime(df_clement['Date'])
df_clement = df_clement.sort_values("Date", ascending=False)


In [41]:
df_clement.head(5)

,Date,Song Title,Artist,Album Title,Listening Time
1740,2024-12-03 18:02:15,Cirice,Ghost,Meliora,53
15427,2024-12-03 16:29:51,I Love Rock 'N Roll,Joan Jett and the Blackhearts,I Love Rock 'N' Roll (Expanded Edition),175
30156,2024-12-03 16:26:56,Wish I Had an Angel,Nightwish,Once,245
31984,2024-12-03 16:22:51,Something To Hide,Grandson,Something To Hide,119
15186,2024-12-03 16:20:42,Monster,PVRIS,Monster,178


# API Request for genres

In [28]:
# Function to fetch tags from Last.fm API
def fetch_tags(title, artist):
    api_key = "e27dd1592fbede0ff3a29b940b5d1935"  # Replace with your Last.fm API key
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "track.getInfo",
        "artist": artist,
        "track": title,
        "api_key": api_key,
        "format": "json"
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            tags = [tag['name'] for tag in data.get('track', {}).get('toptags', {}).get('tag', [])]
            return tags
        else:
            return ["Error: " + response.text]
    except Exception as e:
        return [f"Error: {e}"]

# Step 1: Create a unique DataFrame for Title and Artist
unique_songs = df_clement[["Song Title", "Artist"]].drop_duplicates()

# Step 2: Fetch tags for each unique song
unique_songs["Tags"] = unique_songs.apply(lambda row: fetch_tags(row["Song Title"], row["Artist"]), axis=1)

# Step 3: Merge the tags back to the original DataFrame
res_df = df_clement.merge(unique_songs, on=["Song Title", "Artist"], how="left")

In [31]:
res_df.to_csv("clement_songs_with_tags.csv", index=False)
print("DataFrame exported to songs_with_tags.csv")

DataFrame exported to songs_with_tags.csv


# Import Processed Data

In [3]:
df_clement = pd.read_csv("clement_songs_with_tags_verified.csv")

In [4]:
df_clement['Date'] = pd.to_datetime(df_clement['Date'])
df_clement["Tags"] = df_clement["Tags"].apply(ast.literal_eval)

In [7]:
df_clement

,Date,Song Title,Artist,Album Title,Listening Time,Tags
0,2024-12-03 18:02:15,Cirice,Ghost,Meliora,53,"[heavy metal, doom metal, metal, 2015, hard rock]"
1,2024-12-03 16:29:51,I Love Rock 'N Roll,Joan Jett and the Blackhearts,I Love Rock 'N' Roll (Expanded Edition),175,"[rock, 80s, classic rock, female vocalists, ha..."
2,2024-12-03 16:26:56,Wish I Had an Angel,Nightwish,Once,245,"[symphonic metal, Gothic Metal, metal, Power m..."
3,2024-12-03 16:22:51,Something To Hide,Grandson,Something To Hide,119,[my top songs]
4,2024-12-03 16:20:42,Monster,PVRIS,Monster,178,"[alternative rock, electronic rock, Hip-Hop, e..."
...,...,...,...,...,...,...
36780,2015-06-21 18:56:41,Je garde le sourire,Black M,Le monde plus gros que mes yeux,234,[]
36781,2015-06-21 18:52:46,La légende Black (feat. Dr. Beriz),Black M,Le monde plus gros que mes yeux,229,[]
36782,2015-06-21 18:48:55,C'est tout moi,Black M,Le monde plus gros que mes yeux,358,[]
36783,2015-06-21 18:16:51,"A la vôtre (feat. Jr O Crom, Dry & Big Ali)",Black M,Le monde plus gros que mes yeux,50,[]


## Explore within time interval

In [5]:
# Define the date interval
start_date = "2024-01-01"
end_date = "2025-01-31"

# Filter the DataFrame
date_filtered_df = df_clement[(df_clement['Date'] >= start_date) & (df_clement['Date'] <= end_date)]

## Top Artists (based on listening time)

In [126]:
artist_listening_time = date_filtered_df.groupby('Artist')['Listening Time'].sum().reset_index(name='Total Listening Time')
artist_listening_time.sort_values("Total Listening Time", ascending=False).head(15)


,Artist,Total Listening Time
129,Grandson,43328
141,Imagine Dragons,40106
296,Starset,36294
131,Green Day,18743
308,Sub Urban,17903
312,System of a Down,17703
186,Linkin Park,17161
137,Hollywood Undead,15594
103,Escape the Fate,14923
229,Nirvana,14839


## Top Artists (based on listening count)

In [127]:
artist_listening_count = date_filtered_df.groupby('Artist').size().reset_index(name='Total Artists Listening Count')
artist_listening_count.sort_values("Total Artists Listening Count", ascending=False).head(15)

,Artist,Total Artists Listening Count
141,Imagine Dragons,285
129,Grandson,261
296,Starset,168
186,Linkin Park,138
104,Ethan Bortnick,116
312,System of a Down,115
308,Sub Urban,105
346,Three Days Grace,103
131,Green Day,95
137,Hollywood Undead,94


## Top Genres (Based on Listening Time)

In [130]:
# Step 1: Expand the DataFrame
expanded_df = date_filtered_df.explode("Tags")  # Creates one row per genre

# Step 2: Aggregate by genre
genre_time = expanded_df.groupby("Tags")["Listening Time"].sum().reset_index()

# Step 3: Sort by total listening time
genre_time = genre_time.sort_values(by="Listening Time", ascending=False)

genre_time.head(15)


,Tags,Listening Time
358,rock,337503
146,alternative,230041
151,alternative rock,198391
265,hard rock,93412
334,pop,86592
301,metal,86296
215,electronic,70319
272,indie,61013
190,classic rock,47045
407,vocal,44330


## Top Tracks (Based on Listening Time)

In [132]:
artist_listening_time = date_filtered_df.groupby('Song Title')['Listening Time'].sum().reset_index(name='Total Listening Time')
artist_listening_time.sort_values("Total Listening Time", ascending=False).head(15)

,Song Title,Total Listening Time
208,Holiday / Boulevard of Broken Dreams,10185
321,Monster,9603
326,My Demons,9577
250,It Has Begun,9496
367,Pull Me Under,8727
376,Rain (from The Suicide Squad),8608
52,Blood // Water,8367
437,Sucker (from the series Arcane League of Legends),8283
70,Californication,8062
167,Freak (feat. REI AMI),7356


## Top Tracks (Based on Play Count)

In [133]:
artist_listening_count = date_filtered_df.groupby('Song Title').size().reset_index(name='Total Artists Listening Count')
artist_listening_count.sort_values("Total Artists Listening Count", ascending=False).head(15)

,Song Title,Total Artists Listening Count
546,cut my fingers off,90
173,Gasoline,69
321,Monster,62
77,Carsick,62
484,To Ashes and Blood (from the series Arcane Lea...,47
38,Believer,47
437,Sucker (from the series Arcane League of Legends),46
383,Renegade (We Never Run) (from the series Arcan...,46
405,Savage,44
326,My Demons,43


## Daily Listening Time

In [19]:
df_daily = df_clement.copy()
df_daily['Date'] = df_daily['Date'].dt.date

# Group by Date and calculate total Listening Time
daily_listening_time = df_daily.groupby('Date', as_index=False)['Listening Time'].sum().sort_values('Date', ascending=False)
daily_listening_time.head(15)

,Date,Listening Time
1368,2024-12-03,10212
1367,2024-12-02,566
1366,2024-12-01,12263
1365,2024-11-30,11029
1364,2024-11-29,7703
1363,2024-11-28,1965
1362,2024-11-27,3095
1361,2024-11-26,944
1360,2024-11-22,197
1359,2024-11-21,2417


## Average Monthly Listening Time

In [27]:
# Group by Month and calculate average listening time per day

# Extract year and month
df_clement['Year'] = df_clement['Date'].dt.year
df_clement['Month'] = df_clement['Date'].dt.month_name()

# Group by Year and Month to calculate monthly sums
monthly_sums = df_clement.groupby(['Year', 'Month'], sort=False)['Listening Time'].sum().reset_index()

# Group by Month to calculate the average of monthly sums
average_monthly_sums = monthly_sums.groupby('Month', sort=False)['Listening Time'].mean()

# Ensure months are in calendar order
calendar_order = [
    'January', 'February', 'March', 'April', 'May', 'June', 
    'July', 'August', 'September', 'October', 'November', 'December'
]
average_monthly_sums = average_monthly_sums.reindex(calendar_order)
average_monthly_sums


Month
January      58469.857143
February     55076.333333
March        56511.714286
April        67287.142857
May          55616.857143
June         37841.555556
July         61056.333333
August       36617.375000
September    72816.555556
October      63333.625000
November     56088.888889
December     46058.000000
Name: Listening Time, dtype: float64

## Average Hourly Listening Time

In [6]:
# Extract the hour
date_filtered_df['Hour'] = date_filtered_df['Date'].dt.hour

# Determine the min and max date
min_date = date_filtered_df['Date'].min()
max_date = date_filtered_df['Date'].max()

# Calculate the total number of days (inclusive)
total_days = (max_date - min_date).days + 1

# Create a placeholder for all hours across all days in the observation period
all_hours = pd.DataFrame({'Hour': range(24)})

# Aggregate total listening time per hour
hourly_sums = date_filtered_df.groupby('Hour')['Listening Time'].sum().reset_index()

# Merge to ensure every hour (0-23) is represented
hourly_sums = pd.merge(all_hours, hourly_sums, on='Hour', how='left').fillna(0)

# Compute the average listening time per hour (divide by total days)
hourly_sums['Average Listening Time'] = hourly_sums['Listening Time'] / total_days
hourly_sums

C:\Users\randr\AppData\Local\Temp\ipykernel_15932\707661928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_filtered_df['Hour'] = date_filtered_df['Date'].dt.hour


,Hour,Listening Time,Average Listening Time
0,0,8913,26.526786
1,1,4107,12.223214
2,2,521,1.550595
3,3,46,0.136905
4,4,220,0.654762
5,5,15450,45.982143
6,6,17485,52.038690
7,7,21665,64.479167
8,8,17441,51.907738
9,9,31028,92.345238


---- 

# Export Data

## JSON Structure

In [7]:
json = {
    "users" : [
        {
            "user_id" : "clement",
            "username" : "Clément Laurent",
            "top_artists": [
                
            ],
            "top_genres": [
                
            ],
            "top_tracks": [
                
            ],
            "average_listening_time": {
                "dataMonth": [],
                "dataYear": [],
                "dataDay": []
            }
        }
    ]
}

In [83]:
timespans = [
    ("4 weeks", ('2024-11-22', '2024-12-22')),
    ("3 months", ('2024-09-22', '2024-12-22')),
    ("6 months", ('2024-06-22', '2024-12-22')),
    ("1 year", ('2023-11-22', '2024-12-22')),
    ("all time", ('2002-01-01', '2024-12-22'))
]

In [84]:


for timespan in timespans:
    copy = original_clement.copy()
    
    copy = copy[(copy['Date'] >= timespan[1][0]) & (copy['Date'] <= timespan[1][1])]
    
    df_top_songs = df_top_artist = copy.groupby('Song Title').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_artists = copy.groupby('Artist').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_albums = copy.groupby('Album Title').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False).head(15)
    
    df_top_genres = None
    
    print(df_top_artists, df_top_songs, df_top_albums)
    

('4 weeks', ('2024-11-22', '2024-12-22'))
                       Artist  Total Count
86                       d4vd           21
83                    Woodkid           20
62        Royal & the Serpent           19
27                  FEVER 333           15
6                    Ashnikko           15
82          Twenty One Pilots           14
45                 Mako, Grey           13
84                       Zand           13
50              Misha Mansoor           13
61  Raja Kumari, Stefflon Don           13
4   Arcane, League of Legends           12
28              Freya Ridings           12
71             Stromae, Pomme           12
40              King Princess           12
23                 Eason Chan           11                                            Song Title  Total Count
89  To Ashes and Blood (from the series Arcane Lea...           20
93  Wasteland (from the series Arcane League of Le...           16
35  Hellfire (from the series Arcane League of Leg...           15
57

--- 

# Matt's data


In [56]:
df_matthieu1 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_0.json")
df_matthieu2 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_1.json")
df_matthieu3 = pd.read_json("data/Archive-Matthieu-Spotify/Spotify Account Data/StreamingHistory_music_2.json")

df_matthieu = pd.concat([df_matthieu1, df_matthieu2, df_matthieu3])

In [53]:
df_matthieu["endTime"] = pd.to_datetime(df_matthieu['endTime'])
df_matthieu = df_matthieu.sort_values("endTime", ascending=True)
df_matthieu['msPlayed'] = (df_matthieu["msPlayed"] / 100).astype(int)

In [55]:
df_matthieu.groupby('artistName').size().reset_index(name='Total Count').sort_values('Total Count', ascending=False)

,artistName,Total Count
833,Forrest Frank,888
1836,NF,664
2566,Tiffany Hudson,531
2358,Sofiane Pamart,439
724,Elevation Worship,420
...,...,...
1331,KAMAMILK,1
1330,KAFFKIEZ,1
1329,K3ndrick,1
1328,K2S,1


In [57]:
df_matthieu

,endTime,artistName,trackName,msPlayed
0,2023-09-17 19:40,AJR,Sober Up (feat. Rivers Cuomo),4866
1,2023-12-04 17:07,Lecrae,They Ain’t Know,69652
2,2023-12-05 01:01,gio.,shadows,69115
3,2023-12-05 07:09,Young Oceans,You Are Not Far,134626
4,2023-12-05 07:13,Josiah Queen,Fishes and Loaves,216979
...,...,...,...,...
7807,2024-12-05 23:41,NF,How Could You Leave Us,322586
7808,2024-12-05 23:44,NF,Lie,209213
7809,2024-12-05 23:49,NF,HOPE,264473
7810,2024-12-05 23:53,NF,TRUST,264415


In [124]:
df_clement.dtypes

Date              datetime64[ns]
Song Title                object
Artist                    object
Album Title               object
Listening Time             int64
Tags                      object
dtype: object

---